In [ ]:
import math
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy.special import ndtri
from decimal import Decimal, ROUND_HALF_UP
from sklearn.calibration import calibration_curve
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 200)

In [ ]:
df = pd.read_csv('データ抽出.csv')
df

# データの読み込み

In [ ]:
df = pd.read_csv('データ抽出.csv')
df

In [ ]:
df.info()

# 前処理

In [ ]:
cut = 0

## 治療法

In [ ]:
df['治療法解析用'].value_counts()

In [ ]:
cut += df['治療法解析用'].isnull().sum()
df['治療法解析用'].isnull().sum()

In [ ]:
df = df.dropna(subset=['治療法解析用'])

In [ ]:
df = pd.get_dummies(df, columns=['治療法解析用'], prefix='', prefix_sep='')
df = df.drop(columns='無治療')
df.rename(columns={'化学療法': 'MTA', '放射線治療': 'Radiation'}, inplace=True)
df

## 前回治療からの期間

In [ ]:
cut += df['Last_Treatment'].isnull().sum()
df['Last_Treatment'].isnull().sum()

In [ ]:
df['Last_Treatment'] = df['Last_Treatment'].replace('#NUM!', 0).replace(0, 10000).astype(int)
df['Last_Treatment'].value_counts()

In [ ]:
df['Last_Treatment'] = np.log10(df['Last_Treatment'] + 1)
df['Last_Treatment'].hist()

## 年齢

In [ ]:
cut += df['Age'].isnull().sum()
df['Age'].isnull().sum()

In [ ]:
df = df.dropna(subset=['Age'])

## 性別

In [ ]:
cut += df['Gender'].isnull().sum()
df['Gender'].isnull().sum()

In [ ]:
df['Gender'] = df['Gender'].replace(1,  0).replace(2, 1)
df['Gender'].value_counts()

## BMI

In [ ]:
df['BMI'].isnull().sum()

In [ ]:
df['BMI'].mean()

In [ ]:
df['BMI_NN'] = df['BMI'].fillna(df['BMI'].mean())
df['BMI_NN'].value_counts()

## 手術回数

In [ ]:
cut += df['No_of_Admission'].isnull().sum()
df['No_of_Admission'].isnull().sum()

In [ ]:
df = df.dropna(subset=['No_of_Admission'])
df['No_of_Admission'] = df['No_of_Admission'].astype(int)
df['No_of_Admission'].value_counts()

## 個数

In [ ]:
df['HCC_No'].value_counts()

In [ ]:
cut += df['HCC_No'].isnull().sum()
df['HCC_No'].isnull().sum()

In [ ]:
#df['HCC_No'] = df['HCC_No'].fillna(5).astype(int)
df = df.dropna(subset=['HCC_No'])
df['HCC_No'] = df['HCC_No'].astype(int)
df['HCC_No'].value_counts()

In [ ]:
before = 0
l = []
for i, n in zip(df['Code'], df['HCC_No']):
    if i == before:
        l.append(l[-1] + n)
    else:
        l.append(n)
        before = i

df['No_Cumsum'] = l
df['No_Cumsum'].value_counts()

## サイズ

In [ ]:
df['HCC_size'].value_counts()

In [ ]:
cut += df['HCC_size'].isnull().sum()
df['HCC_size'].isnull().sum()

In [ ]:
df = df.replace('diffuse', '1')
df = df.dropna(subset=['HCC_size'])
df['HCC_size'] = df['HCC_size'].map(lambda x: int(Decimal(str(x)).quantize(Decimal('0'), rounding=ROUND_HALF_UP)))
df['HCC_size'].value_counts()

## サイズ*個数

In [ ]:
df['NoSize'] = df['HCC_No'] * df['HCC_size']
df['NoSize'].value_counts()

In [ ]:
cut += df['NoSize'].isnull().sum()
df['NoSize'].isnull().sum()

In [ ]:
before = 0
l = []
for i, n in zip(df['Code'], df['NoSize']):
    if i == before:
        l.append(l[-1] + n)
    else:
        l.append(n)
        before = i

l_10 = [i//10 for i in l]
df['NoSize_Cumsum'] = l_10
df['NoSize_Cumsum'].value_counts()

## PS

In [ ]:
df['PS'].value_counts()

In [ ]:
#PSは0埋め
df['PS_Raw'] = df['PS'].fillna(0).astype(int)
df['PS_Raw'].value_counts()

## ALBI

In [ ]:
df['ALBI_score'].value_counts()

In [ ]:
cut += df['ALBI_score'].isnull().sum()
df['ALBI_score'].isnull().sum()

In [ ]:
df = df.dropna(subset=['ALBI_score'])
df['ALBI_score'] = df['ALBI_score'].map(lambda x: int(Decimal(str(x*(-100))).quantize(Decimal('0'), rounding=ROUND_HALF_UP)))
df['ALBI_score'].value_counts()

## ALBI_grade

In [ ]:
df['ALBI_grade'].value_counts()

In [ ]:
cut += df['ALBI_grade'].isnull().sum()
df['ALBI_grade'].isnull().sum()

In [ ]:
df['ALBI_grade'] = df['ALBI_grade'].replace('3', '4').replace('2b', '3').replace('2a', '2').astype(int)
df = pd.get_dummies(df, columns=['ALBI_grade'])
df = df.drop(columns='ALBI_grade_1')
df

## AFP

In [ ]:
#cut += df['AFP'].isnull().sum()
df['AFP'].isnull().sum()

In [ ]:
#AFPは0埋め
df['AFP'] = df['AFP'].fillna(0).astype(float)
df.insert(loc=0, column='AFP_100', value= -1)
df.loc[df['AFP'] < 100, 'AFP_100'] = 0
df.loc[~(df['AFP'] < 100), 'AFP_100'] = 1
df['AFP_100'].value_counts()

## L3

In [ ]:
#cut += df['L3'].isnull().sum()
df['L3'].isnull().sum()

In [ ]:
#L3は0埋め
df['L3'] = df['L3'].fillna(0).astype(float)
df.insert(loc=0, column='L3_10', value= -1)
df.loc[df['L3'] < 10, 'L3_10'] = 0
df.loc[~(df['L3'] < 10), 'L3_10'] = 1
df['L3_10'].value_counts()

In [ ]:
df['L3_10'] = df['L3_10'].fillna(0).astype(int)
df['L3_10'].value_counts()

## PIVKA

In [ ]:
#cut += df['PIVKA'].isnull().sum()
df['PIVKA'].isnull().sum()

In [ ]:
#PIVKAは0埋め
df['PIVKA'] = df['PIVKA'].fillna(0).astype(float)
df.insert(loc=0, column='PIVKA_100', value= -1)
df.loc[df['PIVKA'] < 100, 'PIVKA_100'] = 0
df.loc[~(df['PIVKA'] < 100), 'PIVKA_100'] = 1
df['PIVKA_100'].value_counts()

## Vp_grade

In [ ]:
df['Vp_grade'].value_counts()

In [ ]:
cut += df['Vp_grade'].isnull().sum()
df['Vp_grade'].isnull().sum()

In [ ]:
df['Vp_grade'] = df['Vp_grade'].replace(2,  1).replace(3, 1).replace(4, 1)
df['Vp_grade'].value_counts()

## meta_positive

In [ ]:
df['Meta0or1'].value_counts()

In [ ]:
cut += df['Meta0or1'].isnull().sum()
df['Meta0or1'].isnull().sum()

In [ ]:
df = df.dropna(subset=['Meta0or1'])
df['Meta0or1'] = df['Meta0or1'].replace(2, 1).astype(int)
df['Meta0or1'].value_counts()

## etiology

In [ ]:
df['etiology_C1B2BC3Alc4NBNC5'].value_counts()

In [ ]:
cut += df['etiology_C1B2BC3Alc4NBNC5'].isnull().sum()
df['etiology_C1B2BC3Alc4NBNC5'].isnull().sum()

In [ ]:
df = df.dropna(subset=['etiology_C1B2BC3Alc4NBNC5'])
df = df.rename(columns={'etiology_C1B2BC3Alc4NBNC5': 'etiology_class'})
df['etiology_class'] = df['etiology_class'].replace(1,  'C').replace(2, 'B').replace(3, 'BC').replace(4, 'Alc').replace(5, 'NBNC')
df['etiology_class'].value_counts()

In [ ]:
df = pd.get_dummies(df, columns=['etiology_class'])
df.loc[df['etiology_class_BC'] == 1, 'etiology_class_B'] = 1
df.loc[df['etiology_class_BC'] == 1, 'etiology_class_C'] = 1
df = df.drop(columns=['etiology_class_BC', 'etiology_class_NBNC'])
df

## OS

In [ ]:
df['OS_day'] = df['OS_day'].replace('#VALUE!', np.nan).replace('#REF!', np.nan)
cut += df['OS_day'].isnull().sum()
df['OS_day'].isnull().sum()

In [ ]:
df = df.dropna(subset=['OS_day'])
df['OS_day'] = df['OS_day'].astype(int)
df['OS_day'].unique()

## 肝臓がんのみを抽出

In [ ]:
df['肝癌症例'].value_counts()

In [ ]:
cut += len(df[df['肝癌症例']==0])
len(df[df['肝癌症例']==0])

In [ ]:
df = df[df['肝癌症例'] == 1]

## dfとcutの確認

In [ ]:
df

In [ ]:
cut

## Three_Yearの作成

In [ ]:
df['Death1Alive0'] = df['Death1Alive0'].astype(int)
df.insert(loc=0, column='Three_Year', value= -1)
df.loc[(df['OS_day'] < 1095) & (df['Death1Alive0'] == 1), 'Three_Year'] = 0
df.loc[df['OS_day'] >= 1095, 'Three_Year'] = 1
df['Three_Year'].value_counts()

In [ ]:
## 3年後の生死が未確認（OS.day<1095&Death1Alive0=0)を削除
df = df[df['Three_Year'] != -1]

## 学習データ

In [ ]:
data = df.loc[:,['Ablation', 'OPE', 'TAE', 'MTA', 'Radiation', 'Last_Treatment', 'Age', 'Gender', 'BMI_NN', 'No_of_Admission', 'HCC_No', 'No_Cumsum', 
                 'HCC_size', 'NoSize', 'NoSize_Cumsum', 'PS_Raw', 'ALBI_score', 'AFP_100', 'L3_10', 'PIVKA_100', 
                 'Vp_grade', 'Meta0or1', 'etiology_class_C', 'etiology_class_B', 'etiology_class_Alc']]
target = df['Three_Year']
data

In [ ]:
data.dtypes

# TrainとValidの作成

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(data, target, train_size = 0.8, random_state = 5)

In [ ]:
x_train.head()

In [ ]:
corr_inv = np.linalg.inv(np.array(x_train.corr()))
vif = pd.Series(np.diag(corr_inv), index=x_train.columns)
pd.options.display.float_format = '{:.2f}'.format
vif

In [ ]:
x_train = x_train.drop('No_Cumsum', axis=1)
x_train_columns = x_train.columns.values.tolist()
print(x_train_columns)

In [ ]:
train_data = x_train.join(y_train)
train_data

In [ ]:
def stepAIC(descs_l):
    import copy
    descriptors = descs_l
    f_model = smf.glm(formula='Three_Year ~ ' + ' + '.join(descriptors),
                      data=train_data, family=sm.families.Binomial()).fit()
    best_aic = f_model.aic
    best_model = f_model
    while descriptors:
        desc_selected = ''
        flag = 0
        for desk in descriptors:
            used_desks = copy.deepcopy(descriptors)
            used_desks.remove(desk)
            formula = 'Three_Year ~ ' + ' + '.join(used_desks)
            model = smf.glm(formula=formula, 
                            data=train_data, family=sm.families.Binomial()).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_model = model
                desc_selected = desk
                flag = 1
        if flag:
            descriptors.remove(desc_selected)
        else:
            break
    return best_model
 
model = stepAIC(x_train_columns)
print(model.summary().tables[1])

In [ ]:
ddf = pd.DataFrame({'model': model.predict()})
ddf['model_p'] = ddf.model.map(lambda x: 1 if x >= 0.5 else 0)
ddf['exp'] = np.array(train_data.Three_Year)
 
correct_count = len(ddf[ ddf.model_p == ddf.exp ])
print(correct_count)
print(correct_count/len(train_data))

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

cm = confusion_matrix(train_data.Three_Year, ddf.model_p)
cm

In [ ]:
eq = np.zeros(len(train_data))
for i,j in zip(model.params.index, model.params.values):
    if i == 'Intercept':
        eq += np.array([j])
    else:
        eq += j*np.array(train_data[i])
        
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(eq, train_data.Three_Year, marker='o', alpha=0.3)
ax.plot(np.sort(eq), 1/(1+(np.exp(-np.sort(eq)))), 'k-', lw=2)
ax.set_ylabel('Result')

In [ ]:
ddf.exp

In [ ]:
ddf.model_p

In [ ]:
# ROC曲線の値の生成：fpr、tpr、閾値
fpr, tpr, thresholds = roc_curve(ddf.exp, ddf.model)

# ROC曲線のプロット
plt.figure(figsize=(6, 6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
#plt.savefig('results/0111_Test_ROC_3year.jpg', dpi=300)
plt.show()

#AUCの表示
auc_GLM = roc_auc_score(ddf.exp, ddf.model)
print(auc_GLM)

In [ ]:
def roc_auc_ci(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2*AUC**2 / (1 + AUC)
    SE_AUC = math.sqrt((AUC*(1 - AUC) + (N1 - 1)*(Q1 - AUC**2) + (N2 - 1)*(Q2 - AUC**2)) / (N1*N2))
    lower = AUC - 1.96*SE_AUC
    upper = AUC + 1.96*SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower, upper)

roc_auc_ci(ddf.exp, ddf.model)

In [ ]:
def _proportion_confidence_interval(r, n, z):
    A = 2*r + z**2
    B = z*math.sqrt(z**2 + 4*r*(1 - r/n))
    C = 2*(n + z**2)
    return ((A-B)/C, (A+B)/C)

def sensitivity_and_specificity_with_confidence_intervals(TP, FP, FN, TN, alpha):
    
    z = -ndtri((1.0-alpha)/2)
    
    sensitivity_point_estimate = TP/(TP + FN)
    sensitivity_confidence_interval = _proportion_confidence_interval(TP, TP + FN, z)
    
    specificity_point_estimate = TN/(TN + FP)
    specificity_confidence_interval = _proportion_confidence_interval(TN, TN + FP, z)
    
    return sensitivity_point_estimate, specificity_point_estimate, sensitivity_confidence_interval, specificity_confidence_interval

sensitivity_and_specificity_with_confidence_intervals(cm[1][1], cm[0][1], cm[1][0], cm[0][0], 0.95)

In [ ]:
def viz_calibration_curve(y_test, y_pred, name):
    frac_of_pos, mean_pred_value = calibration_curve(y_test, y_pred, n_bins=10)

    fig, ax = plt.subplots(1, 2, figsize=(15,6))
    ax[0].plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    ax[0].plot(mean_pred_value, frac_of_pos, marker="o", label=f'{name}')
    ax[0].set_ylabel("Fraction of positives")
    ax[0].set_ylim([-0.05, 1.05])
    ax[0].legend(loc="lower right")
    ax[0].set_title(f'Calibration plot 1year ({name})')
    
    sns.distplot(y_pred, bins=100, label='predicted score', ax=ax[1])
    ax[1].legend(loc='upper right')
    ax[1].set_xlim([-0.05, 1.05])
    #plt.savefig('results/0111_calibration_3year.jpg', dpi=300)
    plt.show()

# AUCとReliability Diagramの可視化
viz_calibration_curve(ddf.exp, ddf.model, 'GLM')

In [ ]:
#pickle.dump(model, open('models/0111_model_GLM_3year.pickle', 'wb'))